## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-28-10-52-37 +0000,nyt,Live Updates: U.S. Attacks Iran as Trump Calls...,https://www.nytimes.com/live/2026/02/28/world/...
1,2026-02-28-10-48-20 +0000,wapo,Alarms sound across the gulf as Iran threatens...,https://www.washingtonpost.com
2,2026-02-28-10-48-20 +0000,wapo,Live updates: U.S. and Israel launch attack on...,https://www.washingtonpost.com/world/2026/02/2...
3,2026-02-28-10-31-42 +0000,bbc,What we know about the joint US-Israel attack ...,https://www.bbc.com/news/articles/cx2dyz6p3weo...
4,2026-02-28-10-31-08 +0000,nyt,Trump Calls for Overthrow of Iran’s Government,https://www.nytimes.com/2026/02/28/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2519/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,39
3,iran,27
43,warner,18
22,he,18
98,bros,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
122,2026-02-27-23-01-00 +0000,wsj,Trump ends government use of Anthropic’s AI mo...,https://www.wsj.com/politics/policy/trump-will...,172
232,2026-02-27-18-25-13 +0000,nypost,Trump makes sobering admission about Iran talk...,https://nypost.com/2026/02/27/us-news/trump-no...,127
33,2026-02-28-07-10-48 +0000,nypost,"US, Israel launch ‘Operation Epic Fury’ attack...",https://nypost.com/2026/02/28/us-news/us-israe...,116
11,2026-02-28-10-14-09 +0000,nypost,"Read President Trump’s full speech after US, I...",https://nypost.com/2026/02/28/us-news/read-pre...,116
2,2026-02-28-10-48-20 +0000,wapo,Live updates: U.S. and Israel launch attack on...,https://www.washingtonpost.com/world/2026/02/2...,112


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
122,172,2026-02-27-23-01-00 +0000,wsj,Trump ends government use of Anthropic’s AI mo...,https://www.wsj.com/politics/policy/trump-will...
161,88,2026-02-27-21-50-27 +0000,nypost,Warner Bros. Discovery CEO David Zaslav calls ...,https://nypost.com/2026/02/27/business/warner-...
33,77,2026-02-28-07-10-48 +0000,nypost,"US, Israel launch ‘Operation Epic Fury’ attack...",https://nypost.com/2026/02/28/us-news/us-israe...
313,72,2026-02-27-12-00-00 +0000,wsj,Hybrid cyberattacks have become more common. “...,https://www.wsj.com/articles/new-york-state-el...
293,54,2026-02-27-13-19-18 +0000,bbc,Pakistan defence minister says country in 'ope...,https://www.bbc.com/news/articles/cvgjd7pv4y4o...
201,49,2026-02-27-19-54-00 +0000,wsj,The Pentagon said it shot down a “seemingly th...,https://www.wsj.com/politics/national-security...
47,44,2026-02-28-04-07-00 +0000,wsj,Bill Clinton said he had no idea of the crimes...,https://www.wsj.com/politics/house-republicans...
96,37,2026-02-28-00-17-31 +0000,startribune,Charges detail planning of St. Paul anti-ICE c...,https://www.startribune.com/attorney-general-p...
116,37,2026-02-27-23-24-00 +0000,wsj,Kristi Noem for months failed to respond to an...,https://www.wsj.com/politics/national-security...
61,36,2026-02-28-03-04-56 +0000,nypost,Ex-MLB pitcher Dan Serafini gets life in priso...,https://nypost.com/2026/02/27/sports/ex-mlb-pi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
